# FEM-BEM coupling 

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
ball = Sphere(Pnt(0,0,0), 1).bc("fembem")
elec = Cylinder(Pnt(-0.5,0,0), Pnt(0.5,0,0),0.6) * \
    (OrthoBrick(Pnt(-0.3,-1,-1), Pnt(-0.2,1,1))+OrthoBrick(Pnt(0.2,-1,-1), Pnt(0.3,1,1)))
elec.bc("electrode")
ball = ball - elec
geo = CSGeometry()
geo.Add(ball)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

Draw (mesh, clipping={ "pnt" : (0,0,0) })

In [ ]:
order=1
V = H1(mesh,order=order, dirichlet="electrode")
Q = SurfaceL2(mesh, order=order-1)
X = V*Q
(u,lam), (v,mu) = X.TnT()

In [ ]:
f = LinearForm(X)
# f += v(0.7,0,0)  # point source
f.Assemble()

a = BilinearForm(grad(u)*grad(v)*dx - lam*v*ds).Assemble()
b = BilinearForm(grad(u)*grad(v)*dx+u*v*dx + lam*mu*ds).Assemble()
inv = b.mat.Inverse(inverse="sparsecholesky", freedofs=X.FreeDofs())

gf = GridFunction(X)
# gf.vec.data = inv * f.vec
gfu, gflam = gf.components
gfu.Set(IfPos(x, 1, -1), BND)

In [ ]:
Draw (gfu, clipping={ "pnt" : (0,0,0) })

Next, we assemble the corresponding boundary integral operators using Bempp-cl and ngbem

In [ ]:
import ngbem
import bempp.api;
import numpy as np;
import scipy;
import bempp.core

# bempp.api.DEFAULT_DEVICE_INTERFACE="opencl"
bempp.api.DEFAULT_DEVICE_INTERFACE="numba"

In [ ]:
class NGSOperator(BaseMatrix):
    def __init__(self, mat):
        BaseMatrix.__init__(self)
        self.mat = mat
    def IsComplex(self):
        return False
    def Height(self):
        return self.mat.shape[0]
    def Width(self):
        return self.mat.shape[1]
    def CreateRowVector(self):
        return BaseVector(self.Width())
    def CreateColVector(self):
        return BaseVector(self.Height())
    def Mult(self,x,y):
        # y.FV().NumPy()[:] = self.mat * x   
        y.data = self.mat * x     

NGBem implements the trace operator between (in our case) $H^1(\Omega)$ and $H^{1/2}(\partial \Omega)$. It
also provides the mapping between the NGSolve SurfaceL2 and the Bempp-cl space of piecewise constant functions.
In order to make sure that the grids and spaces match, H1_trace also returns a corresponding Bempp-cl function space. (The second parameter makes sure that we only generate the Bempp grid once).

Note that, since we are using piecewise constant functions for $Q$, bnd_trace_matrix will be the identity

In [ ]:
fembem = mesh.Boundaries("fembem")
[bem_c,trace_matrix]=ngbem.ng_to_bempp_trace(V, None, fembem);
[bem_dc, bnd_trace_matrix]=ngbem.ng_to_bempp_trace(Q,bem_c.grid, fembem);

# We will need the following operators:
the single layer operator $V$,
the double layer operator $K$ and a mass matrix $M$

In [ ]:
##set up the bem
bempp.api.VECTORIZATION_MODE = "novec" 
sl=bempp.api.operators.boundary.laplace.single_layer(bem_dc,bem_c,bem_dc) # ,assembler="fmm", device_interface="opencl")
dl=bempp.api.operators.boundary.laplace.double_layer(bem_c,bem_c,bem_dc)#,assembler="fmm", device_interface="opencl")
id_op=bempp.api.operators.boundary.sparse.identity(bem_dc,bem_dc,bem_c)
id_op2=bempp.api.operators.boundary.sparse.identity(bem_c,bem_c,bem_dc)

In [ ]:
embu, emblam = X.embeddings

In [ ]:
bnd_op1=0.5*id_op2 - dl
ngs_bnd1= NGSOperator(bnd_trace_matrix.T) @ NGSOperator(bnd_op1.weak_form()) @ NGSOperator(trace_matrix)
ngs_bnd1= emblam @ ngs_bnd1 @ embu.T

We set up the following block system:
$\begin{pmatrix}
A & M \\
\frac{1}{2} M - K & V
\end{pmatrix} 
\begin{pmatrix}
u \\ \lambda
\end{pmatrix}= \begin{pmatrix} f \\ 0 \end{pmatrix}$

In [ ]:
ngs_sl =  NGSOperator(bnd_trace_matrix.T)  @ NGSOperator(sl.weak_form()) @  NGSOperator(bnd_trace_matrix) 
ngs_sl = emblam @ ngs_sl @ emblam.T   # 1,1 block

In [ ]:
# gfu=GridFunction(X)
# print(a.mat.width)

In [ ]:
lhs=a.mat  + ngs_sl+ ngs_bnd1
# print(lhs.height,f.vec.size,gfu.vec.size)

In [ ]:
ngs_sl.width

In [ ]:
res = f.vec.CreateVector()
res.data = f.vec - lhs*gf.vec
w = gf.vec.CreateVector()
w[:] = 0
solvers.GMRes(A=lhs, b=res, pre=inv, tol=1e-6, printrates=True, x=w,freedofs=X.FreeDofs(),maxsteps=400)
gf.vec.data += w

In [ ]:
gfu, gflam = gf.components
Draw(gfu, clipping = { "pnt" : (0,0,0) })

The product space can provide embedding matrices for the individual components:

$$
E_u = \left( \begin{array}{c} I \\ 0 \end{array} \right)
\qquad
E_\lambda = \left( \begin{array}{c} 0 \\ I \end{array} \right)
$$
